# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Window

In [2]:
# create a Spark session
spark = SparkSession.builder.getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

## Load Data

In [3]:
user_log = spark.read.json('mini_sparkify_event_data.json')

In [4]:
user_log.count()

286500

In [5]:
user_log.toPandas().head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Martha Tilston,Logged In,Colin,M,50,Freeman,277.89016,paid,"Bakersfield, CA",PUT,NextSong,1.538173e+12,29,Rockpools,200,1538352117000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
1,Five Iron Frenzy,Logged In,Micah,M,79,Long,236.09424,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,1.538332e+12,8,Canada,200,1538352180000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9
2,Adam Lambert,Logged In,Colin,M,51,Freeman,282.82730,paid,"Bakersfield, CA",PUT,NextSong,1.538173e+12,29,Time For Miracles,200,1538352394000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
3,Enigma,Logged In,Micah,M,80,Long,262.71302,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,1.538332e+12,8,Knocking On Forbidden Doors,200,1538352416000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9
4,Daft Punk,Logged In,Colin,M,52,Freeman,223.60771,paid,"Bakersfield, CA",PUT,NextSong,1.538173e+12,29,Harder Better Faster Stronger,200,1538352676000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30


In [6]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



## Clean Data

### Check Missing/Null/Empty Values

No NaN values in all columns.

In [7]:
# check: NaN values per column
count_nan_cols = [F.count(F.when(F.isnan(c), c)).alias(c) for c in user_log.columns]
user_log.select(count_nan_cols).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Null values are found in columns related with user information and song information.

In [8]:
# check: null values per column
count_null_cols = [F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in user_log.columns]
user_log.select(count_null_cols).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,58392,0,8346,8346,0,8346,58392,0,8346,0,0,8346,0,58392,0,0,8346,0


Pages that are not `NextSong` will have null values for `artist`, `length` and `song`. 

In [9]:
user_log.filter(F.col('artist').isNull()).toPandas().head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Colin,M,54,Freeman,None,paid,"Bakersfield, CA",PUT,Add to Playlist,1.538173e+12,29,None,200,1538352905000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
1,None,Logged In,Micah,M,84,Long,None,free,"Boston-Cambridge-Newton, MA-NH",GET,Roll Advert,1.538332e+12,8,None,200,1538353150000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9
2,None,Logged In,Micah,M,86,Long,None,free,"Boston-Cambridge-Newton, MA-NH",PUT,Thumbs Up,1.538332e+12,8,None,307,1538353376000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9
3,None,Logged In,Alexi,F,4,Warren,None,paid,"Spokane-Spokane Valley, WA",GET,Downgrade,1.532483e+12,53,None,200,1538354749000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:32.0) G...,54
4,None,Logged In,Alexi,F,7,Warren,None,paid,"Spokane-Spokane Valley, WA",PUT,Thumbs Up,1.532483e+12,53,None,307,1538355255000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:32.0) G...,54


In [10]:
user_log.filter(F.col('artist').isNull()).select('page').dropDuplicates().sort('page').show(30, False)

+-------------------------+
|page                     |
+-------------------------+
|About                    |
|Add Friend               |
|Add to Playlist          |
|Cancel                   |
|Cancellation Confirmation|
|Downgrade                |
|Error                    |
|Help                     |
|Home                     |
|Login                    |
|Logout                   |
|Register                 |
|Roll Advert              |
|Save Settings            |
|Settings                 |
|Submit Downgrade         |
|Submit Registration      |
|Submit Upgrade           |
|Thumbs Down              |
|Thumbs Up                |
|Upgrade                  |
+-------------------------+



In [11]:
user_log.filter(F.col('artist').isNull()!=True).toPandas().head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Martha Tilston,Logged In,Colin,M,50,Freeman,277.89016,paid,"Bakersfield, CA",PUT,NextSong,1538173362000,29,Rockpools,200,1538352117000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
1,Five Iron Frenzy,Logged In,Micah,M,79,Long,236.09424,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,1538331630000,8,Canada,200,1538352180000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9
2,Adam Lambert,Logged In,Colin,M,51,Freeman,282.82730,paid,"Bakersfield, CA",PUT,NextSong,1538173362000,29,Time For Miracles,200,1538352394000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
3,Enigma,Logged In,Micah,M,80,Long,262.71302,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,1538331630000,8,Knocking On Forbidden Doors,200,1538352416000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9
4,Daft Punk,Logged In,Colin,M,52,Freeman,223.60771,paid,"Bakersfield, CA",PUT,NextSong,1538173362000,29,Harder Better Faster Stronger,200,1538352676000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30


In [12]:
user_log.filter(F.col('artist').isNull()!=True).select('page').dropDuplicates().show()

+--------+
|    page|
+--------+
|NextSong|
+--------+



In [13]:
# check: empty values per column
count_invalid_cols = [F.count(F.when(F.col(c)=='', c)).alias(c) for c in user_log.columns]
user_log.select(count_invalid_cols).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8346


Users with empty `userId` are those who did not register and log in.

In [14]:
user_log.filter(F.col('userId')=='').toPandas().head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged Out,None,None,100,None,None,free,None,GET,Home,None,8,None,200,1538355745000,None,
1,None,Logged Out,None,None,101,None,None,free,None,GET,Help,None,8,None,200,1538355807000,None,
2,None,Logged Out,None,None,102,None,None,free,None,GET,Home,None,8,None,200,1538355841000,None,
3,None,Logged Out,None,None,103,None,None,free,None,PUT,Login,None,8,None,307,1538355842000,None,
4,None,Logged Out,None,None,2,None,None,free,None,GET,Home,None,240,None,200,1538356678000,None,


In [15]:
user_log.filter(F.col('userId')=='').groupBy('registration', 'auth').count().toPandas()

,registration,auth,count
0,None,Logged Out,8249
1,None,Guest,97


Since these entries have missing values in many columns, we can drop them from our analysis.

### Drop rows with empty userId

In [16]:
df = user_log.filter(F.col('userId')!='')
df.count()

278154

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

## Define Churn
Use `Cancellation Confirmation` events to define churn.

In [17]:
flag_churn = F.udf(lambda x: 1 if x == 'Cancellation Confirmation' else 0, T.IntegerType())
df = df.withColumn('churn', flag_churn('page'))

## Transform Data

Transform the original dataset (one row per log) to a dataset with user-level information or statistics (one row per user).

### Convert Timestamps (ms) to Datetime

In [18]:
df = df.withColumn('reg_dt', (F.col('registration')/1000).cast(T.TimestampType()))
df = df.withColumn('dt', (F.col('ts')/1000).cast(T.TimestampType()))

### User-based Observation Start/End Date

In [19]:
min_dt = df.agg({'dt':'min'}).collect()[0]['min(dt)']
max_dt = df.agg({'dt':'max'}).collect()[0]['max(dt)']
min_reg_dt = df.agg({'reg_dt':'min'}).collect()[0]['min(reg_dt)']
max_reg_dt = df.agg({'reg_dt':'max'}).collect()[0]['max(reg_dt)']
print(f'min_dt:{min_dt}')
print(f'max_dt:{max_dt}')
print(f'min_reg_dt:{min_reg_dt}')
print(f'max_reg_dt:{max_reg_dt}')

min_dt:2018-10-01 09:01:57
max_dt:2018-12-03 10:11:16
min_reg_dt:2018-03-18 22:44:35
max_reg_dt:2018-11-27 00:49:14


In [20]:
# get first log date
w = Window.partitionBy('userId').orderBy('ts').rangeBetween(Window.unboundedPreceding, Window.unboundedFollowing)
df = df.withColumn('first_dt', F.first('dt').over(w))

In [21]:
# infer observation start date
# - for users with reg_dt < min_dt, use min_dt for obs_start
# - for users with min_dt <= reg_dt < first_dt, use reg_dt for obs_start
# - for users with reg_dt >= first_dt, use first_dt for obs_start
df = df.withColumn('obs_start', 
                   (F.when(F.col('reg_dt')<min_dt, min_dt)
                     .when(F.col('reg_dt')<F.col('first_dt'), F.col('reg_dt'))
                     .otherwise(F.col('first_dt')))
                  )

In [22]:
# infer observation end date
# - for churn users, use churn date for obs_end
# - for non-churn users, use max_dt for obs_end
df = df.withColumn('obs_end', 
                   (F.when(F.last('churn').over(w)==1, F.last('dt').over(w))
                     .otherwise(max_dt))
                  )

In [23]:
# a function to check date-related columns for a user
def display_user(user_id):
    return (df.where(F.col('userId')==user_id)
              .select('userId', 'page', 'reg_dt', 'dt', 'obs_start', 'obs_end')
              .sort('dt')
              .toPandas())

In [24]:
# check: users registered before min_dt
df.filter(F.col('reg_dt')<=min_dt).groupBy('userId').agg({'churn':'max'}).show()

+------+----------+
|userId|max(churn)|
+------+----------+
|100010|         0|
|200002|         0|
|   125|         1|
|    51|         1|
|   124|         0|
|     7|         0|
|    54|         1|
|    15|         0|
|   132|         0|
|100014|         1|
|   101|         1|
|    11|         0|
|   138|         0|
|300017|         0|
|    29|         1|
|    69|         0|
|100021|         1|
|    42|         0|
|   112|         0|
|    87|         1|
+------+----------+
only showing top 20 rows



In [25]:
# a non-churn user
display_user('100010')

,userId,page,reg_dt,dt,obs_start,obs_end
0,100010,NextSong,2018-09-27 11:45:40,2018-10-08 21:58:54,2018-10-01 09:01:57,2018-12-03 10:11:16
1,100010,NextSong,2018-09-27 11:45:40,2018-10-08 22:02:16,2018-10-01 09:01:57,2018-12-03 10:11:16
2,100010,NextSong,2018-09-27 11:45:40,2018-10-08 22:05:32,2018-10-01 09:01:57,2018-12-03 10:11:16
3,100010,Thumbs Up,2018-09-27 11:45:40,2018-10-08 22:05:33,2018-10-01 09:01:57,2018-12-03 10:11:16
4,100010,NextSong,2018-09-27 11:45:40,2018-10-08 22:09:55,2018-10-01 09:01:57,2018-12-03 10:11:16
5,100010,NextSong,2018-09-27 11:45:40,2018-10-08 22:15:11,2018-10-01 09:01:57,2018-12-03 10:11:16
6,100010,NextSong,2018-09-27 11:45:40,2018-10-08 22:18:14,2018-10-01 09:01:57,2018-12-03 10:11:16
7,100010,NextSong,2018-09-27 11:45:40,2018-10-08 22:21:58,2018-10-01 09:01:57,2018-12-03 10:11:16
8,100010,Roll Advert,2018-09-27 11:45:40,2018-10-08 22:22:41,2018-10-01 09:01:57,2018-12-03 10:11:16
9,100010,NextSong,2018-09-27 11:45:40,2018-10-08 22:25:57,2018-10-01 09:01:57,2018-12-03 10:11:16


In [26]:
# a churned user
display_user('125')

,userId,page,reg_dt,dt,obs_start,obs_end
0,125,NextSong,2018-08-02 05:58:59,2018-10-12 13:05:44,2018-10-01 09:01:57,2018-10-12 13:35:18
1,125,NextSong,2018-08-02 05:58:59,2018-10-12 13:11:21,2018-10-01 09:01:57,2018-10-12 13:35:18
2,125,NextSong,2018-08-02 05:58:59,2018-10-12 13:15:11,2018-10-01 09:01:57,2018-10-12 13:35:18
3,125,NextSong,2018-08-02 05:58:59,2018-10-12 13:18:34,2018-10-01 09:01:57,2018-10-12 13:35:18
4,125,NextSong,2018-08-02 05:58:59,2018-10-12 13:22:04,2018-10-01 09:01:57,2018-10-12 13:35:18
5,125,NextSong,2018-08-02 05:58:59,2018-10-12 13:28:35,2018-10-01 09:01:57,2018-10-12 13:35:18
6,125,NextSong,2018-08-02 05:58:59,2018-10-12 13:32:08,2018-10-01 09:01:57,2018-10-12 13:35:18
7,125,NextSong,2018-08-02 05:58:59,2018-10-12 13:35:06,2018-10-01 09:01:57,2018-10-12 13:35:18
8,125,Roll Advert,2018-08-02 05:58:59,2018-10-12 13:35:17,2018-10-01 09:01:57,2018-10-12 13:35:18
9,125,Cancel,2018-08-02 05:58:59,2018-10-12 13:35:18,2018-10-01 09:01:57,2018-10-12 13:35:18


In [27]:
# check: users registered after min_dt
df.filter(F.col('reg_dt')>min_dt).groupBy('userId').agg({'churn':'max'}).show()

+------+----------+
|userId|max(churn)|
+------+----------+
|   155|         0|
|   154|         0|
|   156|         0|
|   153|         0|
|   152|         0|
+------+----------+



In [28]:
# normal case: registration date is before the first log date
# use registration date for obs_start
display_user('156')

,userId,page,reg_dt,dt,obs_start,obs_end
0,156,Home,2018-11-27 00:49:14,2018-11-27 08:14:00,2018-11-27 00:49:14,2018-12-03 10:11:16
1,156,NextSong,2018-11-27 00:49:14,2018-11-27 08:14:02,2018-11-27 00:49:14,2018-12-03 10:11:16
2,156,Roll Advert,2018-11-27 00:49:14,2018-11-27 08:15:10,2018-11-27 00:49:14,2018-12-03 10:11:16
3,156,NextSong,2018-11-27 00:49:14,2018-11-27 08:17:43,2018-11-27 00:49:14,2018-12-03 10:11:16
4,156,Settings,2018-11-27 00:49:14,2018-11-27 08:18:35,2018-11-27 00:49:14,2018-12-03 10:11:16
5,156,NextSong,2018-11-27 00:49:14,2018-11-27 08:21:00,2018-11-27 00:49:14,2018-12-03 10:11:16


In [29]:
# weired case: registration date is after the first log date
# use the first log date for obs_start
display_user('154')

,userId,page,reg_dt,dt,obs_start,obs_end
0,154,Home,2018-11-04 03:00:21,2018-11-03 00:15:35,2018-11-03 00:15:35,2018-12-03 10:11:16
1,154,NextSong,2018-11-04 03:00:21,2018-11-03 00:15:38,2018-11-03 00:15:35,2018-12-03 10:11:16
2,154,NextSong,2018-11-04 03:00:21,2018-11-03 00:18:39,2018-11-03 00:15:35,2018-12-03 10:11:16
3,154,NextSong,2018-11-04 03:00:21,2018-11-03 00:22:19,2018-11-03 00:15:35,2018-12-03 10:11:16
4,154,NextSong,2018-11-04 03:00:21,2018-11-03 00:26:59,2018-11-03 00:15:35,2018-12-03 10:11:16
5,154,NextSong,2018-11-04 03:00:21,2018-11-03 00:32:26,2018-11-03 00:15:35,2018-12-03 10:11:16
6,154,Roll Advert,2018-11-04 03:00:21,2018-11-03 00:33:03,2018-11-03 00:15:35,2018-12-03 10:11:16
7,154,NextSong,2018-11-04 03:00:21,2018-11-03 00:36:29,2018-11-03 00:15:35,2018-12-03 10:11:16
8,154,NextSong,2018-11-04 03:00:21,2018-11-03 00:40:35,2018-11-03 00:15:35,2018-12-03 10:11:16
9,154,Add to Playlist,2018-11-04 03:00:21,2018-11-03 00:40:46,2018-11-03 00:15:35,2018-12-03 10:11:16


### Last Subscription Level

In [30]:
# last subscription level
df = df.withColumn('last_level', F.last('level').over(w))

### Aggregation by User

In [31]:
user_df = df.groupby('userId').agg(
    # User-level features
    F.max('churn').alias('churn'),
    F.first('gender').alias('gender'),
    F.first('reg_dt').alias('reg_dt'),
    F.first('obs_start').alias('obs_start'),
    F.first('obs_end').alias('obs_end'),
    F.first('last_level').alias('last_level'),

    # interaction-level features (exclude Cancellation Confirmation)
    F.count('page').alias('n_act'),
    F.sum(F.when(F.col('page')=='About', 1).otherwise(0)).alias('n_about'),
    F.sum(F.when(F.col('page')=='Add Friend', 1).otherwise(0)).alias('n_addFriend'),
    F.sum(F.when(F.col('page')=='Add to Playlist', 1).otherwise(0)).alias('n_addToPlaylist'),
    F.sum(F.when(F.col('page')=='Cancel', 1).otherwise(0)).alias('n_cancel'),
    F.sum(F.when(F.col('page')=='Downgrade', 1).otherwise(0)).alias('n_downgrade'),
    F.sum(F.when(F.col('page')=='Error', 1).otherwise(0)).alias('n_error'),
    F.sum(F.when(F.col('page')=='Help', 1).otherwise(0)).alias('n_help'),
    F.sum(F.when(F.col('page')=='Home', 1).otherwise(0)).alias('n_home'),
    F.sum(F.when(F.col('page')=='Logout', 1).otherwise(0)).alias('n_logout'),
    F.sum(F.when(F.col('page')=='NextSong', 1).otherwise(0)).alias('n_song'),
    F.sum(F.when(F.col('page')=='Roll Advert', 1).otherwise(0)).alias('n_rollAdvert'),
    F.sum(F.when(F.col('page')=='Save Settings', 1).otherwise(0)).alias('n_saveSettings'),
    F.sum(F.when(F.col('page')=='Settings', 1).otherwise(0)).alias('n_settings'),
    F.sum(F.when(F.col('page')=='Submit Downgrade', 1).otherwise(0)).alias('n_submitDowngrade'),
    F.sum(F.when(F.col('page')=='Submit Upgrade', 1).otherwise(0)).alias('n_submitUpgrade'),
    F.sum(F.when(F.col('page')=='Thumbs Down', 1).otherwise(0)).alias('n_thumbsDown'),
    F.sum(F.when(F.col('page')=='Thumbs Up', 1).otherwise(0)).alias('n_thumbsUp'),
    F.sum(F.when(F.col('page')=='Upgrade', 1).otherwise(0)).alias('n_upgrade'),

    # song-level features
    F.countDistinct('artist').alias('n_artist'),
    F.sum('length').alias('sum_length'),
    
    # session-level features
    F.countDistinct('sessionId').alias('n_session'),
)

In [32]:
# extract new features from some aggregated statistics
user_df = (user_df.withColumn('reg_days', F.datediff('obs_end', 'reg_dt'))
                  .withColumn('obs_hours', (F.unix_timestamp('obs_end')-F.unix_timestamp('obs_start'))/3600)
                  .withColumn('n_act_per_hour', F.col('n_act')/F.col('obs_hours'))
                  .withColumn('n_about_per_hour', F.col('n_about')/F.col('obs_hours'))
                  .withColumn('n_addFriend_per_hour', F.col('n_addFriend')/F.col('obs_hours'))
                  .withColumn('n_addToPlaylist_per_hour', F.col('n_addToPlaylist')/F.col('obs_hours'))
                  .withColumn('n_downgrade_per_hour', F.col('n_downgrade')/F.col('obs_hours'))
                  .withColumn('n_error_per_hour', F.col('n_error')/F.col('obs_hours'))
                  .withColumn('n_help_per_hour', F.col('n_help')/F.col('obs_hours'))
                  .withColumn('n_home_per_hour', F.col('n_home')/F.col('obs_hours'))
                  .withColumn('n_logout_per_hour', F.col('n_logout')/F.col('obs_hours'))
                  .withColumn('n_song_per_hour', F.col('n_song')/F.col('obs_hours'))
                  .withColumn('n_rollAdvert_per_hour', F.col('n_rollAdvert')/F.col('obs_hours'))
                  .withColumn('n_saveSettings_per_hour', F.col('n_saveSettings')/F.col('obs_hours'))
                  .withColumn('n_settings_per_hour', F.col('n_settings')/F.col('obs_hours'))
                  .withColumn('n_submitDowngrade_per_hour', F.col('n_submitDowngrade')/F.col('obs_hours'))
                  .withColumn('n_submitUpgrade_per_hour', F.col('n_submitUpgrade')/F.col('obs_hours'))
                  .withColumn('n_thumbsDown_per_hour', F.col('n_thumbsDown')/F.col('obs_hours'))
                  .withColumn('n_thumbsUp_per_hour', F.col('n_thumbsUp')/F.col('obs_hours'))
                  .withColumn('n_upgrade_per_hour', F.col('n_upgrade')/F.col('obs_hours'))
          )

### Session-level Features

In [33]:
# average number of items per session
avg_session_items_df = (df.groupby('userId', 'sessionId')
                          .agg(F.max('itemInSession').alias('session_items'))
                          .groupby('userId')
                          .agg(F.avg('session_items').alias('avg_session_items')))

In [34]:
# average time(minutes) per session
avg_session_mins_df = (df.groupBy('userId', 'sessionId')
                         .agg(((F.max('ts')-F.min('ts'))/1000/60).alias('session_mins'))
                         .groupBy('userId')
                         .agg(F.avg('session_mins').alias('avg_session_mins')))

In [35]:
# average number of songs per session
avg_session_songs_df = (df.where(F.col('page')=='NextSong')
                          .groupBy('userId', 'sessionId')
                          .agg(F.count('page').alias('session_songs'))
                          .groupBy('userId')
                          .agg(F.avg('session_songs').alias('avg_session_songs')))

In [36]:
# join on userId
user_df = (user_df.join(avg_session_items_df, on='userId')
                  .join(avg_session_mins_df, on='userId')
                  .join(avg_session_songs_df, on='userId'))
assert df.select('userId').dropDuplicates().count() == user_df.count()

In [37]:
# only use these variables
user_df = user_df.select('userId', 'churn', 'gender', 'last_level', 'sum_length', 'n_session', 'reg_days', 'obs_hours', 
                         'n_act_per_hour', 'n_about_per_hour', 'n_addFriend_per_hour', 'n_addToPlaylist_per_hour',
                         'n_cancel', 'n_downgrade_per_hour', 'n_error_per_hour', 'n_help_per_hour',
                         'n_home_per_hour', 'n_logout_per_hour', 'n_song_per_hour', 'n_rollAdvert_per_hour',
                         'n_saveSettings_per_hour', 'n_settings_per_hour', 'n_submitDowngrade_per_hour',
                         'n_submitUpgrade_per_hour', 'n_thumbsDown_per_hour', 'n_thumbsUp_per_hour', 'n_upgrade_per_hour',
                         'avg_session_items', 'avg_session_mins', 'avg_session_songs'
                        )
user_df.printSchema()

root
 |-- userId: string (nullable = true)
 |-- churn: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- last_level: string (nullable = true)
 |-- sum_length: double (nullable = true)
 |-- n_session: long (nullable = false)
 |-- reg_days: integer (nullable = true)
 |-- obs_hours: double (nullable = true)
 |-- n_act_per_hour: double (nullable = true)
 |-- n_about_per_hour: double (nullable = true)
 |-- n_addFriend_per_hour: double (nullable = true)
 |-- n_addToPlaylist_per_hour: double (nullable = true)
 |-- n_cancel: long (nullable = true)
 |-- n_downgrade_per_hour: double (nullable = true)
 |-- n_error_per_hour: double (nullable = true)
 |-- n_help_per_hour: double (nullable = true)
 |-- n_home_per_hour: double (nullable = true)
 |-- n_logout_per_hour: double (nullable = true)
 |-- n_song_per_hour: double (nullable = true)
 |-- n_rollAdvert_per_hour: double (nullable = true)
 |-- n_saveSettings_per_hour: double (nullable = true)
 |-- n_settings_per_hour: double (nul

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.